# Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd 

In [2]:
os.makedirs('LaoDong', exist_ok=True)
os.makedirs('VnExpress', exist_ok=True)
os.makedirs('VTC', exist_ok=True)
os.makedirs('DangCongSan', exist_ok=True)

# LaoDong newspaper

In [ ]:
url = 'https://laodong.vn/'
doc = BeautifulSoup(requests.get(url).text, 'html.parser')
xahoi = doc.find_all(string="Xã hội")
xahoi_tag = xahoi[0].parent
news_topics = xahoi_tag.parent
topics = [a.string for a in news_topics.find_all('a', class_ = 'link')]

max_pages = 150            # get articles from first max_pages pages

# Loop over each class
for topic in topics:

    df = pd.DataFrame()
    atc_title = []
    atc_abstract = []
    atc_source = []
    atc_link = []
    atc_topic = []

    print('Topic: ', topic)
    page_0_url = doc.find_all(string=str(topic))[0].parent['href']
    urls = [str(page_0_url) + '?page=' + str(i) for i in range(1, max_pages + 1)]     # url to each page

    for i in range(len(urls)):
        url = urls[i]
        class_doc = BeautifulSoup(requests.get(url).text, 'html.parser')
        articles = class_doc.find_all('article')                        # get the articles

        print(f'Page: {i}, num_articles: {len(articles)}')
        for article in articles:
            tag = article.find('a', class_ = 'link-title')
            title = tag.string              # article's title
            article_url = tag['href']       # article's url
            article_doc = BeautifulSoup(requests.get(article_url).text, 'html.parser')
            abstraction = article_doc.find('meta', attrs={'name': 'description'})['content']      # article's abstraction

            atc_title.append(str(title))
            atc_abstract.append(abstraction)
            atc_source.append('Báo Lao Động')
            atc_link.append(article_url)
            atc_topic.append(str(topic))

    # construct a dataframe for storing data
    df['title'] = atc_title
    df['abstract'] = atc_abstract
    df['source'] = atc_source
    df['link'] = atc_link
    df['topic'] = atc_topic

    # save a csv file
    file_name = 'LaoDong/' + str(topic) + '.csv'
    df.to_csv(file_name)

In [ ]:
root = 'LaoDong/'
path = [root + topic for topic in os.listdir('LaoDong/')]
all_df = [pd.read_csv(i) for i in path]
pd.concat(all_df).drop_duplicates().to_csv('laodong.csv', index=False)
pd.read_csv('laodong.csv')

In [14]:
pd.read_csv('laodong.csv')

,title,abstract,source,link,topic
0,Ngóng từng ngày điện về khu tái định cư giữa t...,"Gần một năm từ ngày được giao đất về xây nhà,...",Báo Lao Động,https://laodong.vn/ban-doc/ngong-tung-ngay-die...,Bạn đọc
1,Salon 1900 và Mạng lưới ung thư vú Việt Nam dừ...,Theo thông báo trên Fanpage chính thức của Mạ...,Báo Lao Động,https://laodong.vn/ban-doc/salon-1900-va-mang-...,Bạn đọc
2,"Đại diện salon lên tiếng khi bị tố ""ăn bớt"" tó...","Liên quan đến vụ bị tố ""ăn bớt"" tóc dành cho ...",Báo Lao Động,https://laodong.vn/ban-doc/dai-dien-salon-len-...,Bạn đọc
3,Đề nghị bố trí đất ngoài đê sông Hồng làm bãi ...,Cử tri quận Hoàn Kiếm đề nghị TP Hà Nội bố tr...,Báo Lao Động,https://laodong.vn/ban-doc/de-nghi-bo-tri-dat-...,Bạn đọc
4,Sẽ xử lý dứt điểm vụ tranh chấp đất kéo dài hơ...,Đắk Lắk - UBND xã Hòa Phú sẽ giải quyết dứt...,Báo Lao Động,https://laodong.vn/ban-doc/se-xu-ly-dut-diem-v...,Bạn đọc
...,...,...,...,...,...
13050,Thanh niên tử vong tại chỗ sau tiếng nổ lớn tạ...,"Thái Nguyên - Sau tiếng nổ lớn, người dân k...",Báo Lao Động,https://laodong.vn/xa-hoi/thanh-nien-tu-vong-t...,Xã hội
13051,"Sau 1 ngày bị đào trộm, loạt cây xanh trên đườ...",Thái Nguyên - Hơn 20 cây xanh gồm các loạ...,Báo Lao Động,https://laodong.vn/ban-tin/sau-1-ngay-bi-dao-t...,Xã hội
13052,Tây Bắc cuối ngày: Bãi gỗ trái phép trên tuyến...,Bãi gỗ trái phép trên tuyến đường nghìn tỉ l...,Báo Lao Động,https://laodong.vn/video/tay-bac-cuoi-ngay-bai...,Xã hội
13053,"Kênh rạch TPHCM ngập rác sau mưa, cá chết hàng...",TPHCM - Những ngày qua TPHCM xuất hiện mưa ...,Báo Lao Động,https://laodong.vn/photo/kenh-rach-tphcm-ngap-...,Xã hội


# VnExpress newspaper

In [226]:
url = 'https://vnexpress.net/'
doc = BeautifulSoup(requests.get(url).text, 'html.parser')

In [227]:
vn = doc.findAll(title='Thời sự')
vn_root = vn[0].parent.parent
topics = vn_root.find_all('a', title=True,href=True)

In [231]:
href = list([a['href'] for a in topics])
href.remove('/'), 
href.remove('https://video.vnexpress.net')
href.remove('/tin-tuc-24h')
href.remove('/tam-su')
topic_urls = ['https://vnexpress.net' + i for i in href]              # all urls to each topic

In [232]:
topic_names = list([a['title'] for a in topics])       # all topic names
topic_names.remove('Trang chủ')
topic_names.remove('Video')
topic_names.remove('Mới nhất') 
topic_names.remove('Tâm sự')

In [235]:
max_pages = 20            # get articles from first max_pages pages

# Loop over each class
for i in range(len(topic_names)):

    df = pd.DataFrame()
    atc_title = []
    atc_abstract = []
    atc_source = []
    atc_link = []
    atc_topic = []

    print('Topic: ', topic_names[i])
    page_0_url = topic_urls[0]
    urls = [str(page_0_url) + '-p' + str(i) for i in range(1, max_pages + 1)]     # url to each page

    for j in range(len(urls)):
        url = urls[j]
        class_doc = BeautifulSoup(requests.get(url).text, 'html.parser')
        articles = class_doc.findAll('article')                      # get the articles

        print(f'Page: {j+1}, num_articles: {len(articles)}')
        for article in articles:
            flag = article.find('span') is not None
            if flag:
                title = article.find('a',attrs={'data-thumb':True})['title']             # article's title
                article_url = article.find('a',attrs={'data-thumb':True})['href']        # article's url
                article_doc = BeautifulSoup(requests.get(article_url).text, 'html.parser')
                abstraction = article_doc.find('meta', attrs={'name': 'description'})['content']      # article's abstraction

                atc_title.append(str(title))
                atc_abstract.append(abstraction)
                atc_source.append('Báo VnExpress')
                atc_link.append(article_url)
                atc_topic.append(str(topic_names[i]))

    # construct a dataframe for storing data
    df['title'] = atc_title
    df['abstract'] = atc_abstract
    df['source'] = atc_source
    df['link'] = atc_link
    df['topic'] = atc_topic

    # save a csv file
    file_name = 'VnExpress/' + str(topic_names[i]) + '.csv'
    df = df.drop_duplicates()
    df.to_csv(file_name, index=False)

Class:  Thời sự
Page: 1, num_articles: 53
Page: 2, num_articles: 30
Page: 3, num_articles: 30
Page: 4, num_articles: 30
Page: 5, num_articles: 30
Page: 6, num_articles: 30
Page: 7, num_articles: 30
Page: 8, num_articles: 30
Page: 9, num_articles: 30
Page: 10, num_articles: 30
Page: 11, num_articles: 30
Page: 12, num_articles: 30
Page: 13, num_articles: 30
Page: 14, num_articles: 30
Page: 15, num_articles: 30
Page: 16, num_articles: 30
Page: 17, num_articles: 30
Page: 18, num_articles: 30
Page: 19, num_articles: 30
Page: 20, num_articles: 30
Class:  Góc nhìn
Page: 1, num_articles: 53
Page: 2, num_articles: 30
Page: 3, num_articles: 30
Page: 4, num_articles: 30
Page: 5, num_articles: 30
Page: 6, num_articles: 30
Page: 7, num_articles: 30
Page: 8, num_articles: 30
Page: 9, num_articles: 30
Page: 10, num_articles: 30
Page: 11, num_articles: 30
Page: 12, num_articles: 30
Page: 13, num_articles: 30
Page: 14, num_articles: 30
Page: 15, num_articles: 30
Page: 16, num_articles: 30
Page: 17, num

In [ ]:
root = 'VnExpress/'
path = [root + topic for topic in os.listdir('VnExpress/')]
all_df = [pd.read_csv(i) for i in path]
pd.concat(all_df).drop_duplicates().to_csv('vnexpress.csv', index=False)

In [18]:
pd.read_csv('vnexpress.csv')

,title,abstract,source,link,topic
0,Biệt thự của vua Bảo Đại ở Nha Trang được xếp ...,Khánh Hòa- Khu biệt thự cổ Cầu Đá hơn trăm năm...,Báo VnExpress,https://vnexpress.net/biet-thu-cua-vua-bao-dai...,Bất động sản
1,Đưa bé trai Trung Quốc về nước sau 5 năm lưu l...,"Zheng Huanyi, 15 tuổi, bị bố bỏ rơi ở Kiên Gia...",Báo VnExpress,https://vnexpress.net/dua-be-trai-trung-quoc-v...,Bất động sản
2,Doanh nghiệp Nhật muốn xử lý tảo lam ở hồ Xuân...,Công ty Ebis Marine đề xuất vận chuyển máy móc...,Báo VnExpress,https://vnexpress.net/ho-xuan-huong-tao-no-hoa...,Bất động sản
3,Hà Nội dự kiến có hai tuyến đường sắt vành đai,Hà Nội dự kiến được quy hoạch hai tuyến đường ...,Báo VnExpress,https://vnexpress.net/ha-noi-du-kien-co-hai-tu...,Bất động sản
4,Thủ tướng đề nghị Việt - Trung hợp tác xây dựn...,Thủ tướng Phạm Minh Chính hội kiến Chủ tịch Tr...,Báo VnExpress,https://vnexpress.net/thu-tuong-de-nghi-viet-t...,Bất động sản
...,...,...,...,...,...
10760,Hàng trăm hộ dân hiến đất mở rộng gấp đôi hẻm ...,Gần 180 hộ dân tại quận Phú Nhuận hiến đất trị...,Báo VnExpress,https://vnexpress.net/hang-tram-ho-dan-hien-da...,Đời sống
10761,4 dự án nghìn tỷ 'giải cứu' kẹt xe khu đông TP...,"Ngoài hai dự án nút giao An Phú, Mỹ Thuỷ đang ...",Báo VnExpress,https://vnexpress.net/4-du-an-nghin-ty-giai-cu...,Đời sống
10762,Bộ Tài nguyên và Môi trường lý giải vì sao đề ...,Mẫu giấy chứng nhận quyền sử dụng đất mới đảm ...,Báo VnExpress,https://vnexpress.net/bo-tai-nguyen-va-moi-tru...,Đời sống
10763,Công viên phần mềm Đà Nẵng chờ ngày hoạt động,"Được đầu tư gần 1.000 tỷ đồng, công viên phần ...",Báo VnExpress,https://vnexpress.net/cong-vien-phan-mem-da-na...,Đời sống


# VTC newspaper

In [9]:
data = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic"])

u1 = "https://vtc.vn/thoi-su-28/trang-"
u2 = "https://vtc.vn/kinh-te-29/trang-"
u3 = "https://vtc.vn/the-thao-34/trang-"
u4 = "https://vtc.vn/the-gioi-30/trang-"
u5 = "https://vtc.vn/giao-duc-31/trang-"
u6 = "https://vtc.vn/phap-luat-32/trang-"
u7 = "https://vtc.vn/giai-tri-33/trang-"
u8 = "https://vtc.vn/suc-khoe-35/trang-"
u9 = "https://vtc.vn/khoa-hoc-cong-nghe-82/trang-"
u10 = "https://vtc.vn/oto-xe-may-37/trang-"
u11 = "https://vtc.vn/phong-su-kham-pha-36/trang-"
u12 = "https://vtc.vn/song-ket-noi-123.html"

In [10]:
def vtc_topic(web_link):
    if "thoi-su" in web_link:
        return "Thời sự"
    elif "kinh-te" in web_link:
        return "Kinh tế"
    elif "the-thao" in web_link:
        return "Thể thao"
    elif "the-gioi" in web_link:
        return "Thế giới"
    elif "giao-duc" in web_link:
        return "Giáo dục"
    elif "phap-luat" in web_link:
        return "Pháp luật"
    elif "giai-tri" in web_link:
        return "Giải trí"
    elif "suc-khoe" in web_link:
        return "Sức khỏe"
    elif "khoa-hoc-cong-nghe" in web_link:
        return "Khoa học công nghệ"
    elif "oto-xe-may" in web_link:
        return "Xe"
    elif "phong-su-kham-pha" in web_link:
        return "Phóng sự"
    else:
        return "Khám phá"

In [19]:
def find_all_articles_at_vtc(max_sl, web_link, data):
    articles_list = []
    for i in range(1, max_sl + 1):
        url = web_link + str(i) + ".html"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        more_articles = soup.find_all('article', class_='clearfix distance')
        
        for article in more_articles:
            if article.select_one('h3.title-1 a'):
                title = article.select_one('h3.title-1 a')['title']
                link = article.select_one('h3.title-1 a')['href']
                abstract = article.select_one('p').get_text(strip=True)
                articles_list.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link)})

        more_more_articles = soup.find_all('article', class_='distance clearfix')
        
        for article in more_more_articles:
            if article.select_one('h3.title-1 a'):
                title = article.select_one('h3.title-1 a')['title']
                link = article.select_one('h3.title-1 a')['href']
                abstract = article.select_one('p').get_text(strip=True)
                articles_list.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link)})
    
    new_data = pd.DataFrame(articles_list)
    data = pd.concat([data, new_data], ignore_index=True)
    return data

In [ ]:
for u in [u1, u2, u3, u4, u5, u6, u7, u8, u9, u10, u11, u12]:
    data = find_all_articles_at_vtc(50, u, data)

data.drop_duplicates().to_csv('vtc.csv', index=False)

In [22]:
pd.read_csv('vtc.csv')

,title,abstract,source,link,topic
0,Thực hư chuyện kêu gọi 'hùn phước' sau vụ cháy...,Trụ trì chùa Thuyền Lâm (TP Huế) khẳng định kh...,Báo VTC,https://vtc.vn/thuc-hu-chuyen-keu-goi-hun-phuo...,Thời sự
1,"Xử lý loạt vũ trường trá hình, quán karaoke ng...",Cơ quan chức năng huyện Cát Hải (Hải Phòng) xử...,Báo VTC,https://vtc.vn/xu-ly-loat-vu-truong-tra-hinh-q...,Thời sự
2,Dự báo thời tiết ngày mai 25/6: Miền Bắc mưa l...,"Dự báo thời tiết ngày mai 25/6, miền Bắc tiếp ...",Báo VTC,https://vtc.vn/du-bao-thoi-tiet-ngay-mai-25-6-...,Thời sự
3,Phó Thủ tướng chia buồn với thân nhân 3 cháu n...,"Thay mặt Chính phủ, Thủ tướng, Phó Thủ tướng T...",Báo VTC,https://vtc.vn/pho-thu-tuong-chia-buon-voi-tha...,Thời sự
4,"Tăng lương hưu 15%: Đảm bảo công bằng, hài hòa...","Theo đề xuất của Chính phủ, dự kiến từ ngày 1/...",Báo VTC,https://vtc.vn/tang-luong-huu-15-dam-bao-cong-...,Thời sự
...,...,...,...,...,...
9892,Phát hiện giúp con người có thể giao tiếp với ...,Các nhà nghiên cứu cho rằng ngôn ngữ của cá nh...,Báo VTC,https://vtc.vn/phat-hien-giup-con-nguoi-co-the...,Khám phá
9893,Bão Mặt trời mạnh nhất 20 năm vừa tấn công Trá...,Bão Mặt trời mạnh nhất hơn hai thập kỷ đã tấn ...,Báo VTC,https://vtc.vn/bao-mat-troi-manh-nhat-20-nam-v...,Khám phá
9894,Dòng sông bẩn nhất thế giới đã được cứu thế nào?,Các quốc gia tìm mọi cách cứu lấy những dòng s...,Báo VTC,https://vtc.vn/dong-song-ban-nhat-the-gioi-da-...,Khám phá
9895,Hành trình ‘săn’ nhật thực 13 lần trong đời củ...,"Ở tuổi 105, ông Laverne Biser chứng kiến 13 lầ...",Báo VTC,https://vtc.vn/hanh-trinh-san-nhat-thuc-13-lan...,Khám phá


# DangCongSan newspaper

In [ ]:
data = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic"])

In [ ]:
u1 = "https://dangcongsan.vn/thoi-su/p/"
u2 = "https://dangcongsan.vn/lanh-dao-dang-nha-nuoc/p/"
u3 = "https://dangcongsan.vn/xay-dung-dang/p/"
u4 = "https://dangcongsan.vn/tu-tuong-van-hoa/p/"
u5 = "https://dangcongsan.vn/kinh-te/p/"
u6 = "https://dangcongsan.vn/xa-hoi/p/"
u7 = "https://dangcongsan.vn/noi-hay-dung/p/"
u8 = "https://dangcongsan.vn/cung-ban-luan/p/"

In [ ]:
def dcs_topic(web_link):
    if "thoi-su" in web_link:
        return "Thời sự"
    elif "lanh-dao-dang-nha-nuoc" in web_link:
        return "Lãnh đạo Đảng, Nhà nước"
    elif "xay-dung-dang" in web_link:
        return "Xây dựng Đảng"
    elif "tu-tuong-van-hoa" in web_link:
        return "Văn hóa"
    elif "kinh-te" in web_link:
        return "Kinh tế"
    elif "xa-hoi" in web_link:
        return "Xã hội"
    elif "noi-hay-dung" in web_link:
        return "Nơi hay dựng"
    elif "cung-ban-luan" in web_link:
        return "Cùng bàn luận"
    else:
        return "Unknown category"

In [ ]:
def find_all_articles_at_dcs(max_sl, web_link, data):
    articles_list = []
    for i in range(1, max_sl + 1):
        url = web_link + str(i)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        articles = soup.find_all('article')
        
        for article in articles:
            if article.select_one('div.h3.h3cate'):
                title = article.select_one('div.h3.h3cate').get_text(strip=True)
                link = article.select_one('div.col0.box-thumbnail a')['href']
                abstract = article.select_one('p.pctent').get_text(strip=True)
                articles_list.append({"title" : title, "abstract": abstract, "source": "Báo Đảng Cộng Sản", "link" : link, "topic": dcs_topic(web_link)})
    
    new_data = pd.DataFrame(articles_list)
    data = pd.concat([data, new_data], ignore_index=True)
    return data

In [ ]:
for u in [u1, u2, u3, u4, u5, u6, u7, u8]:
    data = find_all_articles_at_dcs(40, u, data)

data.drop_duplicates().to_csv('dcs.csv', index=False)

In [24]:
pd.read_csv('dcs.csv')

,title,abstract,source,link,topic
0,Đề xuất tăng mức giảm trừ gia cảnh khi tăng lương,(ĐCSVN) - Đại biểu Tạ Văn Hạ - Đoàn Đại biểu Q...,Báo Đảng Cộng Sản,https://dangcongsan.vn/xa-hoi/de-xuat-tang-muc...,Thời sự
1,Hà Tĩnh - Khăm Muồn (Lào) đẩy mạnh quan hệ hữu...,(ĐCSVN) - Để các nội dung ký kết hợp tác giữa ...,Báo Đảng Cộng Sản,https://dangcongsan.vn/doi-ngoai/ha-tinh-kham-...,Thời sự
2,"Cập nhật “địa chỉ số” của thửa đất, tài sản gắ...","(ĐCSVN) - ""Bộ Tài nguyên và Môi trường và Bộ X...",Báo Đảng Cộng Sản,https://dangcongsan.vn/thoi-su/cap-nhat-dia-ch...,Thời sự
3,Vụ cháy tại Trung Kính do chập mạch điện đầu x...,"(ĐCSVN) - Vụ cháy xảy ra tại nhà số 1, ngách 4...",Báo Đảng Cộng Sản,https://dangcongsan.vn/thoi-su/vu-chay-tai-tru...,Thời sự
4,"Tạo điều kiện để người dân tiếp cận, sử dụng n...",(ĐCSVN) - Đại biểu Lã Thanh Tân - Đoàn ĐBQH TP...,Báo Đảng Cộng Sản,https://dangcongsan.vn/y-te/tao-dieu-kien-de-n...,Thời sự
...,...,...,...,...,...
8817,Kính cẩn nghiêng mình trước sự hy sinh của các...,"(ĐCSVN) - Ai đó đã viết rằng, trong thời bình,...",Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/kinh-can-...,Cùng bàn luận
8818,Vì cuộc đời không có chữ “nếu”…,(ĐCSVN) – Ước mơ đỗ Đại học Y Hà Nội của Ngô M...,Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/vi-cuoc-d...,Cùng bàn luận
8819,Ngành thuế nỗ lực đảm bảo nguồn thu ngân sách ...,(ĐCSVN) - Để hỗ trợ người nộp thuế khôi phục h...,Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/nganh-thu...,Cùng bàn luận
8820,Dự báo thị trường bất động sản phải chăng được...,"(ĐCSVN) – Thời điểm này, đã thấy dấu hiệu tái ...",Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/du-bao-th...,Cùng bàn luận


# DanTri newspaper

In [4]:
DanTri_topics = ['kinh-doanh',
                 'xa-hoi',
                 'the-gioi',
                 'giai-tri',
                 'bat-dong-san',
                 'the-thao',
                 'lao-dong-viec-lam',
                 'suc-khoe',
                 'o-to-xe-may',
                 'suc-manh-so',
                 'giao-duc',
                 'an-sinh',
                 'phap-luat',
                 'du-lich',
                 'doi-song',
                 'tinh-yeu-gioi-tinh',
                 'khoa-hoc-cong-nghe']
DanTri_topics_names = {'kinh-doanh': 'Kinh doanh',
                       'xa-hoi': 'Xã hội',
                       'the-gioi': 'Thế giới',
                       'giai-tri': 'Giải trí',
                       'bat-dong-san': 'Bất động sản',
                       'the-thao': 'Thể thao',
                       'lao-dong-viec-lam': 'Việc làm',
                       'suc-khoe': 'Sức khỏe',
                       'o-to-xe-may': 'Xe ++',
                       'suc-manh-so': 'Sức mạnh số',
                       'giao-duc': 'Giáo dục',
                       'an-sinh': 'An sinh',
                       'phap-luat': 'Pháp luật',
                       'du-lich': 'Du lịch',
                       'doi-song': 'Đời sống',
                       'tinh-yeu-gioi-tinh': 'Tình yêu',
                       'khoa-hoc-cong-nghe': 'Khoa học - Công nghệ'}

print( len(DanTri_topics) )
print( DanTri_topics_names[ DanTri_topics[0] ] )

17
Kinh doanh


In [5]:
def get_abstract_DanTri(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.content, "html.parser")

    if soup.select_one('h2.singular-sapo'):
        abstract = soup.select_one('h2.singular-sapo').get_text(strip=True)
    else:
        abstract = None

    return abstract

In [ ]:
def find_all_articles_at_DanTri(max_sl, topic, data):

  for i in range(1, max_sl + 1):
    url = 'https://dantri.com.vn/' + topic + '/trang-' + str(i) + '.htm'
    response = requests.get(url)
    print(url, response)

    soup = BeautifulSoup(response.content, "html.parser")

    more_articles = soup.find_all('article', class_='article-item')

    for article in more_articles:

      if article.select_one('h3.article-title a'):
        title = article.select_one('h3.article-title a').get_text(strip=True)

        link_ = article.select_one('h3.article-title a')['href']
        link = 'https://dantri.com.vn/' + link_

        abstract = get_abstract_DanTri(link)

        artical_info = {
            "title" : title,
            "abstract": abstract,
            "source": "Báo Dân Trí",
            "link" : link,
            "topic": DanTri_topics_names[topic],
        }

        data.append( artical_info )
      else:
        continue
  print()

  return data

In [ ]:
data_DanTri = []

for topic in DanTri_topics:
    data_DanTri = find_all_articles_at_DanTri( 10, topic, data_DanTri )
    print()

https://dantri.com.vn/kinh-doanh/trang-1.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-2.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-3.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-4.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-5.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-6.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-7.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-8.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-9.htm <Response [200]>
https://dantri.com.vn/kinh-doanh/trang-10.htm <Response [200]>


https://dantri.com.vn/xa-hoi/trang-1.htm <Response [200]>
https://dantri.com.vn/xa-hoi/trang-2.htm <Response [200]>
https://dantri.com.vn/xa-hoi/trang-3.htm <Response [200]>
https://dantri.com.vn/xa-hoi/trang-4.htm <Response [200]>
https://dantri.com.vn/xa-hoi/trang-5.htm <Response [200]>
https://dantri.com.vn/xa-hoi/trang-6.htm <Response [200]>
https://dantri.com.vn/xa-hoi/

In [ ]:
df_DanTri = pd.DataFrame(data_DanTri)
df_DanTri = df_DanTri.drop_duplicates( subset = 'title', keep = 'last')
df_DanTri.to_csv('DanTri.csv', index = False)

In [13]:
pd.read_csv('dantri.csv')

,title,abstract,source,link,topic
0,"Cấm vợ, chồng, anh chị em ruột tham gia đấu gi...","(Dân trí) - Luật sửa đổi, bổ sung một số điều ...",Báo Dân Trí,https://dantri.com.vn//kinh-doanh/cam-vo-chong...,Kinh doanh
1,"Giá vàng thế giới giảm sâu, giá USD tự do cao ...",(Dân trí) - Giá vàng trong nước đứng im. Vàng ...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/gia-vang-the...,Kinh doanh
2,"Vàng nhẫn giảm sâu, giá USD tự do lập đỉnh kỷ ...",(Dân trí) - Giá vàng miếng SJC không thay đổi ...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/vang-nhan-gi...,Kinh doanh
3,Chính phủ kiến nghị cho phép sử dụng 18.220 tỷ...,(Dân trí) - Ủy ban Tài chính Ngân sách kiến ng...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/chinh-phu-ki...,Kinh doanh
4,"Giá vàng đứng im 17 phiên liên tiếp, giá thế g...",(Dân trí) - Giá vàng thế giới giảm sâu do chịu...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/gia-vang-dun...,Kinh doanh
...,...,...,...,...,...
6303,"Ngựa từng có 4 ngón chân, tại sao chúng biến mất?",(Dân trí) - Ngựa đã trải qua quá trình tiến hó...,Báo Dân Trí,https://dantri.com.vn//khoa-hoc-cong-nghe/ngua...,Khoa học - Công nghệ
6304,Internet lượng tử đạt được cột mốc quan trọng,(Dân trí) - Các nhà vật lý tại Đại học Stony B...,Báo Dân Trí,https://dantri.com.vn//khoa-hoc-cong-nghe/inte...,Khoa học - Công nghệ
6305,Hơn 90 đề tài ấn tượng được trình bày tại Hội ...,(Dân trí) - Những dự án nghiên cứu triển vọng ...,Báo Dân Trí,https://dantri.com.vn//khoa-hoc-cong-nghe/hon-...,Khoa học - Công nghệ
6306,"Trung Quốc chế tạo pin từ canxi: Rẻ hơn, bền h...",(Dân trí) - Nghiên cứu được kỳ vọng sẽ mở đườn...,Báo Dân Trí,https://dantri.com.vn//khoa-hoc-cong-nghe/trun...,Khoa học - Công nghệ


# All data

In [10]:
news = ['dantri.csv','dcs.csv','laodong.csv','vnexpress.csv','vtc.csv']
df = pd.concat([pd.read_csv(i) for i in news])
df.dropna().to_csv('Articles.csv', index=False)

In [11]:
pd.read_csv('Articles.csv')

,title,abstract,source,link,topic
0,"Cấm vợ, chồng, anh chị em ruột tham gia đấu gi...","(Dân trí) - Luật sửa đổi, bổ sung một số điều ...",Báo Dân Trí,https://dantri.com.vn//kinh-doanh/cam-vo-chong...,Kinh doanh
1,"Giá vàng thế giới giảm sâu, giá USD tự do cao ...",(Dân trí) - Giá vàng trong nước đứng im. Vàng ...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/gia-vang-the...,Kinh doanh
2,"Vàng nhẫn giảm sâu, giá USD tự do lập đỉnh kỷ ...",(Dân trí) - Giá vàng miếng SJC không thay đổi ...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/vang-nhan-gi...,Kinh doanh
3,Chính phủ kiến nghị cho phép sử dụng 18.220 tỷ...,(Dân trí) - Ủy ban Tài chính Ngân sách kiến ng...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/chinh-phu-ki...,Kinh doanh
4,"Giá vàng đứng im 17 phiên liên tiếp, giá thế g...",(Dân trí) - Giá vàng thế giới giảm sâu do chịu...,Báo Dân Trí,https://dantri.com.vn//kinh-doanh/gia-vang-dun...,Kinh doanh
...,...,...,...,...,...
48352,Phát hiện giúp con người có thể giao tiếp với ...,Các nhà nghiên cứu cho rằng ngôn ngữ của cá nh...,Báo VTC,https://vtc.vn/phat-hien-giup-con-nguoi-co-the...,Khám phá
48353,Bão Mặt trời mạnh nhất 20 năm vừa tấn công Trá...,Bão Mặt trời mạnh nhất hơn hai thập kỷ đã tấn ...,Báo VTC,https://vtc.vn/bao-mat-troi-manh-nhat-20-nam-v...,Khám phá
48354,Dòng sông bẩn nhất thế giới đã được cứu thế nào?,Các quốc gia tìm mọi cách cứu lấy những dòng s...,Báo VTC,https://vtc.vn/dong-song-ban-nhat-the-gioi-da-...,Khám phá
48355,Hành trình ‘săn’ nhật thực 13 lần trong đời củ...,"Ở tuổi 105, ông Laverne Biser chứng kiến 13 lầ...",Báo VTC,https://vtc.vn/hanh-trinh-san-nhat-thuc-13-lan...,Khám phá
